# Let's install the Following :

In [2]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB

# Import

In [3]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import warnings
import textwrap

# Importing API Keys and PATHS

In [4]:
with open('config/openai_key.json') as f:
    config = json.load(f)

API_KEY = config.get('API_KEY')
openai.api_key = API_KEY

ARTICLE = 'https://newsinfo.inquirer.net/1997710/storm-kong-rey-moving-faster-maintains-strength-in-approaching-par'

In [5]:
# Wraps the text so it doesnt overflow
def print_log(log_text, width=80):
    wrapped_text = textwrap.fill(log_text, width=width)
    print(wrapped_text)

In [6]:
prompt = '''
You are a news summarization assistant, skilled at distilling news articles into concise, clear summaries. The user will provide news articles, and your task is to produce a high-quality summary for each article. Follow these structured guidelines:

1. Identify Core Elements
Main Topic: Determine the overarching subject or event the article covers (e.g., a new policy, a scientific discovery, a global event).
Key Points: Focus on the essential facts, including:
Who is involved (e.g., key figures, organizations, or countries).
What happened (describe the primary event or announcement).
Where it took place, if relevant.
When it occurred, especially if it’s time-sensitive.
Why it’s important or how it impacts the audience or stakeholders.
2. Summarize Effectively
Conciseness: Aim for a brief but comprehensive summary that captures the essence without unnecessary detail. Summaries should generally be around 2–5 sentences unless the user specifies otherwise.
Clarity: Use simple and direct language, avoiding jargon or overly complex terms to keep the summary accessible.
Prioritize Information: Highlight the most critical elements and exclude minor details, anecdotal information, or any content that doesn’t contribute directly to the main points.
3. Maintain Objectivity
Neutral Tone: Report the facts as presented in the article without adding your interpretation or opinions.
Avoid Speculation: Stick to information clearly stated in the article, and avoid assuming outcomes or implications beyond what’s written.
4. Preserve Context and Original Intent
Reflect the Original Message: Ensure that the summary captures the original purpose and tone of the article.
Accuracy: Use the same terminology for key names, events, or policies as the article to maintain accuracy and avoid misinterpretation.
Contextual Cues: If relevant, add brief context to help readers understand the importance (e.g., “following recent industry shifts”).
5. Adjust for User-Specific Requests
Flexibility: Be prepared to adapt the summary length or focus if the user requests additional details on certain aspects or a more in-depth overview.
Supplementary Information: If the user needs, offer additional context on recurring news topics or background to clarify complex subjects.



'''

In [7]:
input = '''
Storm Kong-rey accelerates, keeps strength as it approaches PAR
By: Jason Sigales - Reporter / @jasonsigalesINQ
INQUIRER.net / 11:37 AM October 26, 2024
Storm Kong-rey accelerates, keeps strength as it approaches PAR
Tropical Storm Kong-rey, which will be given the local name Leon once it enters the Philippine area of responsibility (PAR), increases speed while maintaining its strength as it continues its westward path on Saturday morning, October 26, 2024, according to the Philippine Atmospheric, Geophysical and Astronomical Services Administration (Pagasa). The storm may enter the PAR either on Saturday night or early Sunday morning (October 27), but it was predicted to be far from the Philippine landmass. Weather satellite image from Pagasa

MANILA, Philippines — Tropical Storm Kong-rey, which will be given the local name Leon once it enters the Philippine area of responsibility (PAR), has picked up speed while maintaining its strength as it continues its westward path.

The Philippine Atmospheric, Geophysical and Astronomical Services Administration (Pagasa) said the storm may enter the PAR either on Saturday night or early Sunday morning, but it also predicted that Kong-rey will be far from the Philippine landmass.

READ: Tropical storm Kong-rey outside PAR maintains strength

FEATURED STORIES
NEWSINFO
DBM OKs earlier release of gov’t workers’ yearend bonus, cash gift
NEWSINFO
Tropical Storm Kong-rey to enter PAR on Sunday, to be named Leon
NEWSINFO
Kristine death toll hits over 40 as damage widens
In its 11 a.m. bulletin, Pagasa said Tropical Storm Kong-rey was currently moving at a pace of 30 kilometers per hour (kph) westward – faster than the 25 kph it posted on Friday evening. The tropical storm was located 1,630 kilometers east of Central Luzon as of 10 a.m. on Saturday, still packing maximum sustained winds of 65 kph near the center and gustiness of up to 80 kph.

Pagasa said this tropical storm may develop into a severe tropical storm by Sunday and further intensify into a typhoon by Monday, October 28.
'''

In [8]:
struc = [{'role':'system','content': prompt}]
struc.append({'role':'user','content':input})
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struc)
response = chat.choices[0].message.content
print_log("Assistant: " + response, width=100)

Assistant: Tropical Storm Kong-rey, expected to be named Leon upon entering the Philippine area of
responsibility (PAR), is accelerating westward at 30 kilometers per hour while maintaining maximum
sustained winds of 65 kph and gusts up to 80 kph. According to the Philippine Atmospheric,
Geophysical and Astronomical Services Administration (Pagasa), the storm is anticipated to enter the
PAR late Saturday or early Sunday, October 27, but will be far from the Philippine landmass. There
is potential for Kong-rey to develop into a severe tropical storm by Sunday and strengthen into a
typhoon by Monday, October 28.
